# 📚 ISEF Paper Scraper - Perianal Crohn's Disease

**Purpose:** Download research papers from PubMed Central for systematic review.

**Strategy:** Broad search queries + strict local relevance filter.

**Output:** PDFs saved to Google Drive in cluster subfolders.

---

## Cell 1: Setup & Dependencies

In [ ]:
# Install BioPython for Entrez API access
!pip install biopython -q

# Mount Google Drive for persistent storage
from google.colab import drive
drive.mount('/content/drive')

print("✅ Setup complete!")

## Cell 2: Configuration

In [ ]:
import os
from pathlib import Path

# ============================================================================
# CONFIGURATION - EDIT THESE IF NEEDED
# ============================================================================

# Google Drive save location
SAVE_DIR = Path("/content/drive/MyDrive/ISEF_Papers")

# Entrez API credentials
ENTREZ_EMAIL = "johs03047@gmail.com"
ENTREZ_API_KEY = "e6dbad0565fd2136d119de63f233dbd92108"

# Search settings
RETMAX = 500  # Papers per search term
RATE_LIMIT = 10  # Requests per second (max with API key)

# Create directory structure
SAVE_DIR.mkdir(parents=True, exist_ok=True)
(SAVE_DIR / "Biologics").mkdir(exist_ok=True)
(SAVE_DIR / "Surgical").mkdir(exist_ok=True)
(SAVE_DIR / "Regenerative").mkdir(exist_ok=True)
(SAVE_DIR / "General").mkdir(exist_ok=True)

print(f"📁 Save directory: {SAVE_DIR}")
print(f"📧 Email: {ENTREZ_EMAIL}")
print(f"🔑 API Key: {ENTREZ_API_KEY[:10]}...")
print(f"📊 RetMax: {RETMAX} papers/term")
print("✅ Configuration complete!")

## Cell 3: Paper Scraper Engine

In [ ]:
import json
import re
import time
import logging
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Set
from dataclasses import dataclass, field, asdict
import threading
import requests
from Bio import Entrez

# ============================================================================
# SETUP ENTREZ
# ============================================================================

Entrez.email = ENTREZ_EMAIL
Entrez.api_key = ENTREZ_API_KEY

REQUEST_INTERVAL = 1.0 / RATE_LIMIT
_last_request_time = 0
_rate_lock = threading.Lock()

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s'
)
logger = logging.getLogger(__name__)

# ============================================================================
# SEARCH TERMS WITH CLUSTER ASSIGNMENT
# ============================================================================

SEARCH_TERMS = [
    # Biologics cluster
    {'query': '("Infliximab" AND "Fistula")', 'cluster': 'Biologics'},
    {'query': '("Adalimumab" AND "Fistula")', 'cluster': 'Biologics'},
    {'query': '("Ustekinumab" AND "Fistula")', 'cluster': 'Biologics'},
    {'query': '("Vedolizumab" AND "Fistula")', 'cluster': 'Biologics'},

    # Surgical cluster
    {'query': '("Seton" AND "Fistula")', 'cluster': 'Surgical'},
    {'query': '("Fistulotomy" AND "Fistula")', 'cluster': 'Surgical'},
    {'query': '("LIFT" AND "Fistula")', 'cluster': 'Surgical'},
    {'query': '("VAAFT" AND "Fistula")', 'cluster': 'Surgical'},
    {'query': '("Advancement Flap" AND "Fistula")', 'cluster': 'Surgical'},

    # Regenerative cluster
    {'query': '("Stem Cells" AND "Fistula")', 'cluster': 'Regenerative'},
    {'query': '("Darvadstrocel" AND "Fistula")', 'cluster': 'Regenerative'},
    {'query': '("Fibrin Glue" AND "Fistula")', 'cluster': 'Regenerative'},

    # General/broad cluster
    {'query': '("Perianal" AND "Crohn")', 'cluster': 'General'},
    {'query': '("Anal Fistula" AND "Treatment")', 'cluster': 'General'},
    {'query': '("Fistula-in-ano" AND "MRI")', 'cluster': 'General'},
]

# ============================================================================
# RELEVANCE FILTER
# ============================================================================

LOCATION_KEYWORDS = {
    "perianal", "peri-anal", "peri anal",
    "anal", "rectovaginal", "anorectal",
    "ano-rectal", "recto-vaginal",
    "anoperineal", "ano-perineal",
    "perineal", "cryptoglandular",
    "fistula-in-ano", "fistula in ano"
}

PATHOLOGY_KEYWORDS = {
    "fistula", "fistulae", "fistulizing", "fistulising",
    "crohn", "crohn's", "crohns",
    "ibd", "inflammatory bowel",
    "abscess", "abscesses",
    "pfcd"
}


def is_relevant(text: str) -> Tuple[bool, Dict[str, List[str]]]:
    """Returns True if text contains BOTH location AND pathology keywords."""
    if not text:
        return False, {"location": [], "pathology": []}

    text_lower = text.lower()

    location_matches = [kw for kw in LOCATION_KEYWORDS if kw in text_lower]
    pathology_matches = [kw for kw in PATHOLOGY_KEYWORDS if kw in text_lower]

    is_rel = len(location_matches) > 0 and len(pathology_matches) > 0
    return is_rel, {"location": location_matches, "pathology": pathology_matches}


# ============================================================================
# DATA CLASSES
# ============================================================================

@dataclass
class PRISMALog:
    start_time: str = ""
    end_time: str = ""
    search_terms: Dict[str, Dict] = field(default_factory=dict)
    totals: Dict[str, int] = field(default_factory=lambda: {
        "total_found": 0,
        "unique_papers": 0,
        "relevant": 0,
        "downloaded": 0,
        "skipped_irrelevant": 0,
        "skipped_no_pdf": 0,
        "skipped_error": 0,
        "duplicates_removed": 0
    })
    errors: List[Dict] = field(default_factory=list)


# ============================================================================
# RATE LIMITER
# ============================================================================

def rate_limited_request():
    """Ensures we don't exceed rate limit."""
    global _last_request_time
    with _rate_lock:
        now = time.time()
        elapsed = now - _last_request_time
        if elapsed < REQUEST_INTERVAL:
            time.sleep(REQUEST_INTERVAL - elapsed)
        _last_request_time = time.time()


# ============================================================================
# PMC FUNCTIONS
# ============================================================================

def search_pmc(query: str, retmax: int = RETMAX) -> Tuple[List[str], int]:
    """Search PMC directly for free full text papers."""
    rate_limited_request()

    try:
        handle = Entrez.esearch(
            db="pmc",
            term=query,
            retmax=retmax,
            sort="relevance"
        )
        record = Entrez.read(handle)
        handle.close()

        pmcids = record.get("IdList", [])
        total_count = int(record.get("Count", 0))

        logger.info(f"Query: {query[:50]}... -> {total_count} total, {len(pmcids)} returned")
        return pmcids, total_count

    except Exception as e:
        logger.error(f"Search error: {e}")
        return [], 0


def fetch_pmc_details(pmcids: List[str]) -> List[Dict]:
    """Fetch metadata for PMC articles."""
    if not pmcids:
        return []

    all_papers = []
    batch_size = 50

    for i in range(0, len(pmcids), batch_size):
        batch = pmcids[i:i + batch_size]
        rate_limited_request()

        try:
            handle = Entrez.efetch(
                db="pmc",
                id=",".join(batch),
                rettype="xml",
                retmode="xml"
            )
            content = handle.read()
            handle.close()

            papers = parse_pmc_xml(content, batch)
            all_papers.extend(papers)

        except Exception as e:
            logger.error(f"Fetch error for batch {i//batch_size + 1}: {e}")
            # Fallback to individual fetches
            for pmcid in batch:
                paper = fetch_single_pmc(pmcid)
                if paper:
                    all_papers.append(paper)

    return all_papers


def fetch_single_pmc(pmcid: str) -> Optional[Dict]:
    """Fetch single PMC article metadata."""
    rate_limited_request()

    try:
        handle = Entrez.esummary(db="pmc", id=pmcid)
        record = Entrez.read(handle)
        handle.close()

        if record:
            doc = record[0]
            return {
                "pmcid": f"PMC{pmcid}",
                "title": doc.get("Title", ""),
                "authors": doc.get("AuthorList", []),
                "first_author": doc.get("AuthorList", ["Unknown"])[0].split()[0] if doc.get("AuthorList") else "Unknown",
                "year": str(doc.get("PubDate", ""))[:4],
                "journal": doc.get("Source", ""),
                "abstract": "",
                "doi": doc.get("DOI", "")
            }
    except Exception as e:
        pass

    return None


def parse_pmc_xml(xml_content: bytes, pmcids: List[str]) -> List[Dict]:
    """Parse PMC XML response."""
    papers = []

    try:
        content = xml_content.decode('utf-8', errors='ignore')
        articles = re.split(r'<article[^>]*>', content)

        for i, article in enumerate(articles[1:], 1):
            try:
                paper = {}

                # PMC ID
                pmcid_match = re.search(r'<article-id pub-id-type="pmc">(\d+)</article-id>', article)
                paper["pmcid"] = f"PMC{pmcid_match.group(1)}" if pmcid_match else f"PMC{pmcids[i-1] if i-1 < len(pmcids) else ''}"

                # Title
                title_match = re.search(r'<article-title[^>]*>(.+?)</article-title>', article, re.DOTALL)
                paper["title"] = re.sub(r'<[^>]+>', '', title_match.group(1)).strip() if title_match else ""

                # Authors
                authors = []
                author_matches = re.findall(r'<surname>([^<]+)</surname>\s*<given-names>([^<]+)</given-names>', article)
                for surname, given in author_matches:
                    authors.append(f"{surname} {given}")
                paper["authors"] = authors
                paper["first_author"] = authors[0].split()[0] if authors else "Unknown"

                # Year
                year_match = re.search(r'<year>(\d{4})</year>', article)
                paper["year"] = year_match.group(1) if year_match else "Unknown"

                # Journal
                journal_match = re.search(r'<journal-title[^>]*>([^<]+)</journal-title>', article)
                paper["journal"] = journal_match.group(1).strip() if journal_match else ""

                # Abstract
                abstract_match = re.search(r'<abstract[^>]*>(.+?)</abstract>', article, re.DOTALL)
                if abstract_match:
                    paper["abstract"] = re.sub(r'<[^>]+>', ' ', abstract_match.group(1)).strip()
                    paper["abstract"] = re.sub(r'\s+', ' ', paper["abstract"])
                else:
                    paper["abstract"] = ""

                # DOI
                doi_match = re.search(r'<article-id pub-id-type="doi">([^<]+)</article-id>', article)
                paper["doi"] = doi_match.group(1) if doi_match else ""

                # Keywords
                keywords = re.findall(r'<kwd>([^<]+)</kwd>', article)
                paper["keywords"] = keywords

                if paper["pmcid"] and (paper["title"] or paper["abstract"]):
                    papers.append(paper)

            except Exception as e:
                continue

    except Exception as e:
        logger.error(f"XML parse error: {e}")

    return papers


def get_pmc_pdf_url(pmcid: str) -> Optional[str]:
    """Get PDF download URL from PMC."""
    pmc_num = pmcid.replace("PMC", "")
    rate_limited_request()

    try:
        url = f"https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id=PMC{pmc_num}"
        response = requests.get(url, timeout=30)

        if response.status_code == 200:
            content = response.text

            pdf_match = re.search(r'href="([^"]+\.pdf)"', content)
            if pdf_match:
                pdf_url = pdf_match.group(1)
                if pdf_url.startswith("ftp://"):
                    pdf_url = pdf_url.replace("ftp://ftp.ncbi.nlm.nih.gov", "https://ftp.ncbi.nlm.nih.gov")
                return pdf_url

        return f"https://www.ncbi.nlm.nih.gov/pmc/articles/PMC{pmc_num}/pdf/"

    except Exception as e:
        return f"https://www.ncbi.nlm.nih.gov/pmc/articles/PMC{pmc_num}/pdf/"


def download_pdf(url: str, save_path: Path) -> Tuple[bool, Optional[str]]:
    """Download PDF from URL."""
    rate_limited_request()

    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36"
        }

        response = requests.get(url, headers=headers, timeout=60, stream=True, allow_redirects=True)

        if response.status_code == 200:
            content_type = response.headers.get("Content-Type", "").lower()

            if "pdf" in content_type or url.endswith(".pdf") or response.content[:4] == b'%PDF':
                save_path.parent.mkdir(parents=True, exist_ok=True)

                with open(save_path, "wb") as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)

                if save_path.stat().st_size > 5000:
                    return True, None
                else:
                    save_path.unlink(missing_ok=True)
                    return False, "File too small"
            else:
                return False, f"Not PDF: {content_type[:30]}"
        else:
            return False, f"HTTP {response.status_code}"

    except requests.Timeout:
        return False, "Timeout"
    except Exception as e:
        return False, str(e)[:50]


# ============================================================================
# PROGRESS SAVING
# ============================================================================

PROGRESS_FILE = SAVE_DIR / "download_progress.json"
METADATA_FILE = SAVE_DIR / "paper_metadata.json"
LOG_FILE = SAVE_DIR / "download_log.json"


def save_progress(prisma_log: PRISMALog, all_metadata: List[Dict], seen_pmcids: Set[str]):
    """Save progress for resumability."""
    progress = {
        "prisma_log": asdict(prisma_log),
        "seen_pmcids": list(seen_pmcids),
        "paper_count": len(all_metadata)
    }
    with open(PROGRESS_FILE, "w") as f:
        json.dump(progress, f, indent=2)

    with open(METADATA_FILE, "w") as f:
        json.dump(all_metadata, f, indent=2)


def load_progress() -> Tuple[Optional[PRISMALog], Set[str], List[Dict]]:
    """Load previous progress if exists."""
    if not PROGRESS_FILE.exists():
        return None, set(), []

    try:
        with open(PROGRESS_FILE, "r") as f:
            progress = json.load(f)

        log_data = progress.get("prisma_log", {})
        prisma_log = PRISMALog(
            start_time=log_data.get("start_time", ""),
            end_time=log_data.get("end_time", ""),
            search_terms=log_data.get("search_terms", {}),
            totals=log_data.get("totals", PRISMALog().totals),
            errors=log_data.get("errors", [])
        )

        seen_pmcids = set(progress.get("seen_pmcids", []))

        metadata = []
        if METADATA_FILE.exists():
            with open(METADATA_FILE, "r") as f:
                metadata = json.load(f)

        logger.info(f"✅ Resumed from progress: {len(seen_pmcids)} papers seen")
        return prisma_log, seen_pmcids, metadata

    except Exception as e:
        logger.warning(f"Could not load progress: {e}")
        return None, set(), []


# ============================================================================
# MAIN PIPELINE
# ============================================================================

def sanitize_filename(name: str) -> str:
    """Create safe filename."""
    name = re.sub(r'[<>:"/\\|?*\'\[\]()]', '', name)
    name = re.sub(r'\s+', '_', name)
    return name[:40]


def process_search_term(term_info: Dict, prisma_log: PRISMALog, seen_pmcids: Set[str],
                        all_metadata: List[Dict]) -> int:
    """Process a single search term."""

    query = term_info['query']
    cluster = term_info['cluster']
    term_name = query.replace('"', '').replace('(', '').replace(')', '')[:40]

    if term_name in prisma_log.search_terms:
        logger.info(f"⏭️ Skipping already processed: {term_name}")
        return 0

    term_log = {
        "query": query,
        "cluster": cluster,
        "found": 0,
        "new": 0,
        "relevant": 0,
        "downloaded": 0,
        "skipped_irrelevant": 0,
        "skipped_duplicate": 0,
        "skipped_error": 0
    }

    # Search PMC
    pmcids, total_count = search_pmc(query)
    term_log["found"] = total_count
    prisma_log.totals["total_found"] += len(pmcids)

    if not pmcids:
        prisma_log.search_terms[term_name] = term_log
        return 0

    # Remove duplicates
    new_pmcids = [p for p in pmcids if f"PMC{p}" not in seen_pmcids]
    term_log["skipped_duplicate"] = len(pmcids) - len(new_pmcids)
    term_log["new"] = len(new_pmcids)
    prisma_log.totals["duplicates_removed"] += term_log["skipped_duplicate"]

    logger.info(f"  📄 New papers: {len(new_pmcids)} (skipped {term_log['skipped_duplicate']} duplicates)")

    if not new_pmcids:
        prisma_log.search_terms[term_name] = term_log
        return 0

    # Fetch metadata
    papers = fetch_pmc_details(new_pmcids)

    downloaded = 0
    cluster_dir = SAVE_DIR / cluster

    for paper in papers:
        pmcid = paper.get("pmcid", "")
        if not pmcid:
            continue

        seen_pmcids.add(pmcid)
        prisma_log.totals["unique_papers"] += 1

        # Combine text for relevance check
        text_to_check = " ".join([
            paper.get("title", ""),
            paper.get("abstract", ""),
            " ".join(paper.get("keywords", []))
        ])

        # RELEVANCE FILTER
        is_rel, matches = is_relevant(text_to_check)

        if not is_rel:
            term_log["skipped_irrelevant"] += 1
            prisma_log.totals["skipped_irrelevant"] += 1
            continue

        term_log["relevant"] += 1
        prisma_log.totals["relevant"] += 1

        # Generate filename
        safe_author = sanitize_filename(paper.get("first_author", "Unknown"))
        year = paper.get("year", "Unknown")
        safe_query = sanitize_filename(term_name.split("AND")[0])
        filename = f"{safe_author}_{year}_{safe_query}_{pmcid}.pdf"
        pdf_path = cluster_dir / filename

        # Skip if already downloaded
        if pdf_path.exists() and pdf_path.stat().st_size > 5000:
            paper["pdf_path"] = str(pdf_path)
            paper["status"] = "exists"
            paper["cluster"] = cluster
            paper["relevance_matches"] = matches
            paper["search_term"] = query
            all_metadata.append(paper)
            term_log["downloaded"] += 1
            downloaded += 1
            continue

        # Get PDF URL and download
        pdf_url = get_pmc_pdf_url(pmcid)

        if pdf_url:
            success, error = download_pdf(pdf_url, pdf_path)

            if success:
                paper["pdf_path"] = str(pdf_path)
                paper["status"] = "downloaded"
                paper["cluster"] = cluster
                paper["relevance_matches"] = matches
                paper["search_term"] = query
                all_metadata.append(paper)
                term_log["downloaded"] += 1
                prisma_log.totals["downloaded"] += 1
                downloaded += 1
                logger.info(f"    ✅ {filename[:50]}")
            else:
                paper["status"] = "error"
                paper["error"] = error
                paper["cluster"] = cluster
                paper["search_term"] = query
                all_metadata.append(paper)
                term_log["skipped_error"] += 1
                prisma_log.totals["skipped_error"] += 1
        else:
            term_log["skipped_error"] += 1
            prisma_log.totals["skipped_no_pdf"] += 1

    prisma_log.search_terms[term_name] = term_log

    # Save progress after each term
    save_progress(prisma_log, all_metadata, seen_pmcids)

    return downloaded


def main():
    """Main execution."""
    print("""
╔══════════════════════════════════════════════════════════════════════╗
║       📚 ISEF PAPER SCRAPER - PERIANAL CROHN'S DISEASE 📚            ║
╠══════════════════════════════════════════════════════════════════════╣
║  Strategy: Broad PMC search + strict relevance filter               ║
║  Search Terms: 15 queries across 4 clusters                         ║
║  RetMax: 500 per term                                               ║
║  Output: Google Drive (cluster subfolders)                          ║
╚══════════════════════════════════════════════════════════════════════╝
    """)

    # Try to resume from progress
    prisma_log, seen_pmcids, all_metadata = load_progress()

    if prisma_log is None:
        prisma_log = PRISMALog(start_time=datetime.now().isoformat())
        seen_pmcids = set()
        all_metadata = []
        print("🆕 Starting fresh run...")
    else:
        print(f"🔄 Resuming from {len(seen_pmcids)} previously seen papers...")

    total_downloaded = 0

    # Process each search term
    for i, term_info in enumerate(SEARCH_TERMS, 1):
        print(f"\n{'='*70}")
        print(f"[{i}/{len(SEARCH_TERMS)}] {term_info['query']} -> {term_info['cluster']}")
        print(f"{'='*70}")

        try:
            downloaded = process_search_term(term_info, prisma_log, seen_pmcids, all_metadata)
            total_downloaded += downloaded
            print(f"  📥 Downloaded {downloaded} papers from this query")
        except Exception as e:
            logger.error(f"Error processing {term_info['query']}: {e}")
            continue

    # Finalize
    prisma_log.end_time = datetime.now().isoformat()

    # Save final log
    with open(LOG_FILE, "w") as f:
        json.dump(asdict(prisma_log), f, indent=2)

    # Save final metadata
    with open(METADATA_FILE, "w") as f:
        json.dump(all_metadata, f, indent=2)

    # Print summary
    print("\n" + "="*70)
    print("📊 FINAL SUMMARY - PRISMA FLOW")
    print("="*70)
    print(f"\n{'Metric':<45} {'Count':>10}")
    print("-"*57)
    print(f"{'Total records found in PMC':<45} {prisma_log.totals['total_found']:>10}")
    print(f"{'Unique papers (after dedup)':<45} {prisma_log.totals['unique_papers']:>10}")
    print(f"{'Passed relevance filter':<45} {prisma_log.totals['relevant']:>10}")
    print(f"{'Successfully downloaded':<45} {prisma_log.totals['downloaded']:>10}")
    print("-"*57)
    print(f"{'Skipped - irrelevant':<45} {prisma_log.totals['skipped_irrelevant']:>10}")
    print(f"{'Skipped - duplicates':<45} {prisma_log.totals['duplicates_removed']:>10}")
    print(f"{'Skipped - download error':<45} {prisma_log.totals['skipped_error']:>10}")
    print("-"*57)

    print("\n\n📁 BY CLUSTER:")
    print("-"*70)
    cluster_counts = {}
    for term_name, term_data in prisma_log.search_terms.items():
        cluster = term_data.get("cluster", "Unknown")
        if cluster not in cluster_counts:
            cluster_counts[cluster] = 0
        cluster_counts[cluster] += term_data.get("downloaded", 0)

    for cluster, count in cluster_counts.items():
        print(f"  {cluster}: {count} PDFs")

    print("\n" + "="*70)
    print(f"📁 Output directory: {SAVE_DIR}")
    print(f"📋 PRISMA log: {LOG_FILE}")
    print(f"📋 Metadata: {METADATA_FILE}")
    print("="*70 + "\n")

    print("✅ COMPLETE!")

    return prisma_log


print("✅ Engine loaded! Run main() in the next cell.")

## Cell 4: Run the Scraper

In [ ]:
# Run the paper scraper!
# This will:
# 1. Search PMC for 15 broad queries
# 2. Filter papers by relevance (must mention anal/perianal + fistula/Crohn)
# 3. Download PDFs to Google Drive in cluster folders
# 4. Save progress after each term (resumable if interrupted)

prisma_log = main()

## Cell 5: Check Results (Optional)

In [ ]:
# Check what was downloaded
import os

print("📁 Downloaded PDFs by cluster:\n")

for cluster in ["Biologics", "Surgical", "Regenerative", "General"]:
    cluster_path = SAVE_DIR / cluster
    if cluster_path.exists():
        pdfs = list(cluster_path.glob("*.pdf"))
        print(f"{cluster}: {len(pdfs)} PDFs")
        for pdf in pdfs[:3]:  # Show first 3
            print(f"  - {pdf.name[:60]}...")
        if len(pdfs) > 3:
            print(f"  ... and {len(pdfs) - 3} more")
        print()

# Total count
total_pdfs = len(list(SAVE_DIR.glob("**/*.pdf")))
print(f"\n📊 TOTAL PDFs: {total_pdfs}")